<a href="https://colab.research.google.com/github/shawn-kg/Projects-in-Machine-Learning-and-AI/blob/main/Homework2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 2: Ensemble Learning


The file used for this project was heart.csv which can be found in the git repository. 

# Task 1: Decision Tree Classifier

In [2]:
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from sklearn import tree
from sklearn.tree import export_text
import graphviz

col_names = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target']
heart = pd.read_csv('drive/MyDrive/heart.csv',header=None, names=col_names)


feature_cols = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']

X = heart[feature_cols]
y = heart.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training and 30% test


# Initialize our decision tree object
classification_tree = tree.DecisionTreeClassifier()

# Train our decision tree (tree induction + pruning)
classification_tree = classification_tree.fit(X_train,y_train)

y_pred = classification_tree.predict(X_test)


print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Precision:",metrics.precision_score(y_test,y_pred))
print("Recall:",metrics.recall_score(y_test,y_pred))

dot_data = tree.export_graphviz(classification_tree, out_file=None) 
graph = graphviz.Source(dot_data) 
graph.render("iris") 

# Initialize our decision tree object
classification_tree = tree.DecisionTreeClassifier()

classification_tree.max_depth = 4;
# Train our decision tree (tree induction + pruning)
classification_tree = classification_tree.fit(X_train,y_train)

y_pred = classification_tree.predict(X_test)


dot_data = tree.export_graphviz(classification_tree, out_file=None) 
graph = graphviz.Source(dot_data) 
graph.render("iris2") 

print("\nAccuracy  with depth of 4:",metrics.accuracy_score(y_test, y_pred))
print("Precision with depth of 4:",metrics.precision_score(y_test,y_pred))
print("Recall with depth of 4:",metrics.recall_score(y_test,y_pred))

Accuracy: 0.7582417582417582
Precision: 0.8181818181818182
Recall: 0.72

Accuracy  with depth of 4: 0.7362637362637363
Precision with depth of 4: 0.76
Recall with depth of 4: 0.76


The parameter I tried to change in reference to the Decision Tree Classifier was the maximum depth of the tree. The original depth of the tree without setting it explicitly was 10 while I reduced it to 4 to test the difference that parameter would make. It seems that reducing the depth to 4 reduced the accuracy and precision of the model overall. However the recall seemed to rise when I decreased the depth of the tree. Recall is important for our data since we are trying to predict heart disease in patients with certain features. Since recall measures the amount of correct predictions we had for those who were actually diagnosed with heard disease it would be important for our model to get correct since we would want to reduce the amount of false negatives. 

# Task 2: Bagging and Boosting 

In [27]:
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

#Bagging Random Forest Algorithm
model = RandomForestClassifier(max_depth=5)
cv = RepeatedStratifiedKFold(n_splits=10,n_repeats=3, random_state=1)
n_scores = cross_val_score(model,X,y,scoring= 'accuracy',cv=cv,n_jobs=-1,error_score='raise')
o_scores = cross_val_score(model,X,y,scoring= 'precision',cv=cv,n_jobs=-1,error_score='raise')
p_scores = cross_val_score(model,X,y,scoring = 'recall', cv = cv,n_jobs=-1,error_score='raise')

print('Random Forest Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))
print('Random Forest Precision: %.3f (%.3f)' % (mean(o_scores), std(n_scores)))
print('Random Forest Recall: %.3f (%.3f)' % (mean(p_scores), std(p_scores)))



#Boosting Gradient Boost Algorithm
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=5, random_state=0)
q_scores = cross_val_score(clf,X,y,scoring= 'accuracy',cv=cv,n_jobs=-1,error_score='raise')
r_scores = cross_val_score(clf,X,y,scoring= 'precision',cv=cv,n_jobs=-1,error_score='raise')
s_scores = cross_val_score(clf,X,y,scoring = 'recall', cv = cv,n_jobs=-1,error_score='raise')

print('\nGradient Boosting Classifier Accuracy: %.3f (%.3f)' % (mean(q_scores), std(q_scores)))
print('Gradient Boosting Classifier Precision: %.3f (%.3f)' % (mean(r_scores), std(r_scores)))
print('Gradient Boosting Classifier Recall: %.3f (%.3f)' % (mean(s_scores), std(s_scores)))


Random Forest Accuracy: 0.822 (0.065)
Random Forest Precision: 0.821 (0.065)
Random Forest Recall: 0.886 (0.077)

Gradient Boosting Classifier Accuracy: 0.780 (0.076)
Gradient Boosting Classifier Precision: 0.801 (0.089)
Gradient Boosting Classifier Recall: 0.810 (0.105)


For a bagging algorithm I utilized the Random Forest Algorithm and for a boosting algorithm I utilized the Gradient Boost Algorithm. In all cases of accuracy, precision, and recall the Random Forest Algorithm performed better overall by a tiny factor. This does make sense however since random forest is a bagging algorithm that applies the same model to different bootstrap samples. This means that it is probably more likely to overfit the data than the Gradient Boosting Classifier that improves weaker models to stronger models as it goes on applying the algorithm. However something that was great overall is that in each classifier the highest metric was the recall which is what we are really targeting for our heart disease prediction. 

# Task 3: Effectiveness of the Three Models


Though I had the program output the accuracy, precision, and recall metrics, our model should really look to optimize the recall metric. Recall is used to see of all the positive cases that are actually diagnosed how many did our model predict correctly? This is very important for this model because more than anything we want to make sure we are diagnosing all those who actually have heart disease correctly. When you think about it, a false positive is not terrible because over time you could realize that the patient has no symptoms and the patient is going to be continuously seen by a doctor. However a false negative is very bad because a patient is likely to not see a doctor if they have no reason to believe they have heart disease and would not get the correct treatment. Maximizing recall really allows us to reduce the number of false negatives we may have. In our three algorithms above, the random forest algorithm is what gave us the best results concerning recall with a measure of 0.886 while the gradient boost algorithm gave us a recall of 0.810 and the vanilla decision tree classifier gave us a recall of 0.72 at max depth and 0.76 at a depth of 4. A choice of a different metric would change our results in how we choose the depth of our decision trees. If we wanted to maximize accuracy and not just recall we would really want to increase the depths of our trees a bit more to the point where we would not overfit but not underfit the data as well. This is easily demonstrated from our vanilla decision tree classifier where a decrease in depth gave us a higher recall but a smaller accuracy. Overall for the needs of those who would utilize this model, we would want to use the random forest classifier since it seems the most adept at maximizing recall. 